In [19]:
prompt = """
- Which role of the teacher would answer the following question better?
- The answer should be either 'math teacher' or 'history teacher' with no additional words.

Case 1
Question : what is the Newton’s law?
Answer : math teacher

Case 2 
Question : How did World War I end?
Answer : history teacher

Question : 
"""


In [25]:
import requests
import json

def send_chat_request(model, message_content):
    url = 'http://localhost:11434/api/chat'
    payload = {
        "model": model,
        "messages": [
            { "role": "user", "content": message_content }
        ]
    }

    response = requests.post(url, json=payload)
    
    response_lines = response.text.split('\n')
    contents = []
    
    for line in response_lines:
        if line.strip(): 
            try:
                message = json.loads(line)
                contents.append(message['message']['content'])
            except json.JSONDecodeError:
                print(f"Failed to decode JSON: {line}")
            except KeyError:
                print(f"Key error in line: {line}")
    
    concatenated_content = ''.join(contents)
    return concatenated_content

def make_question(question):
    return prompt + question

llama3 = "llama3"
question_1 = make_question("What is the Pythagorean theorem?")
question_2 = make_question("What were the main causes of the French Revolution?")

llama_answer_1 = send_chat_request(llama3, question_1)
llama_answer_2 = send_chat_request(llama3, question_2)
print(llama_answer_1)
print(llama_answer_2)

Math teacher
History teacher


In [26]:
phi3 = "phi3"
phi3_answer_1 = send_chat_request(llama3, question_1)
phi3_answer_2 = send_chat_request(llama3, question_2)
print(phi3_answer_1)
print(phi3_answer_2)

Math teacher
History teacher


In [36]:
def parse_response(s):
    s = s.strip('[]')
    number_strings = s.split(',')

    number_list = [int(num.strip()) for num in number_strings]

    return number_list

prompts = [
    "Please sort the following list of numbers in ascending order. List: [5, 3, 8, 1]",
    "Sort the following numbers in ascending order. Example: [4, 2, 9, 7] -> [2, 4, 7, 9] List: [5, 3, 8, 1]",
    "Here is how to sort numbers in ascending order:\n1. Select the first element of the list.\n2. Compare it with the remaining elements.\n3. Place the smaller elements before it.\n4. Repeat the process.\n\nNow, sort the following list: [5, 3, 8, 1]"
]

test_cases = [
    ([5, 3, 8, 1], [1, 3, 5, 8]),
    ([2, 1, 4, 3], [1, 2, 3, 4]),
    ([10, 9, 8, 7], [7, 8, 9, 10]),
    ([3, 6, 2, 7], [2, 3, 6, 7]),
    ([1, 5, 3, 2], [1, 2, 3, 5])
]

model = "llama3"

results = []
for prompt in prompts:
    success_count = 0
    for input_list, expected_output in test_cases:
        prompt_with_list = "- The answer should be number list format with no additional words. (example answer : [1, 2, 3, 4]\n" + prompt.replace("[5, 3, 8, 1]", str(input_list))
        response_text = send_chat_request(model, prompt_with_list)
        sorted_list = parse_response(response_text)
        if sorted_list == expected_output:
            success_count += 1
    success_rate = success_count / len(test_cases) * 100
    results.append((prompt, success_count, success_rate))

# Print the results
for idx, (prompt, success_count, success_rate) in enumerate(results):
    print(f"Prompt {idx + 1}:")
    print(f"Prompt Text: {prompt}")
    print(f"Success Count: {success_count}/{len(test_cases)}")
    print(f"Success Rate: {success_rate}%")
    print()

Prompt 1:
Prompt Text: Please sort the following list of numbers in ascending order. List: [5, 3, 8, 1]
Success Count: 5/5
Success Rate: 100.0%

Prompt 2:
Prompt Text: Sort the following numbers in ascending order. Example: [4, 2, 9, 7] -> [2, 4, 7, 9] List: [5, 3, 8, 1]
Success Count: 5/5
Success Rate: 100.0%

Prompt 3:
Prompt Text: Here is how to sort numbers in ascending order:
1. Select the first element of the list.
2. Compare it with the remaining elements.
3. Place the smaller elements before it.
4. Repeat the process.

Now, sort the following list: [5, 3, 8, 1]
Success Count: 5/5
Success Rate: 100.0%

